# Wstępne obrabianie danych



---



In [ ]:
#dodawanie bibliotek
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from collections import defaultdict
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
from sklearn import manifold

from tensorflow import keras
from tensorflow.keras import layers
import tensorflow as tf
from sklearn.metrics import r2_score

%matplotlib inline

In [ ]:

data = pd.read_csv('avocado.csv')
data.T

,0,1,2,3,4,5,6,7,8,9,...,18239,18240,18241,18242,18243,18244,18245,18246,18247,18248
Unnamed: 0,0,1,2,3,4,5,6,7,8,9,...,2,3,4,5,6,7,8,9,10,11
Date,2015-12-27,2015-12-20,2015-12-13,2015-12-06,2015-11-29,2015-11-22,2015-11-15,2015-11-08,2015-11-01,2015-10-25,...,2018-03-11,2018-03-04,2018-02-25,2018-02-18,2018-02-11,2018-02-04,2018-01-28,2018-01-21,2018-01-14,2018-01-07
AveragePrice,1.33,1.35,0.93,1.08,1.28,1.26,0.99,0.98,1.02,1.07,...,1.56,1.54,1.57,1.56,1.57,1.63,1.71,1.87,1.93,1.62
Total Volume,64236.62,54876.98,118220.22,78992.15,51039.6,55979.78,83453.76,109428.33,99811.42,74338.76,...,22128.42,17393.3,18421.24,17597.12,15986.17,17074.83,13888.04,13766.76,16205.22,17489.58
4046,1036.74,674.28,794.7,1132.0,941.48,1184.27,1368.92,703.75,1022.15,842.4,...,2162.67,1832.24,1974.26,1892.05,1924.28,2046.96,1191.7,1191.92,1527.63,2894.77
4225,54454.85,44638.81,109149.67,71976.41,43838.39,48067.99,73672.72,101815.36,87315.57,64757.44,...,3194.25,1905.57,2482.65,1928.36,1368.32,1529.2,3431.5,2452.79,2981.04,2356.13
4770,48.16,58.33,130.5,72.58,75.78,43.61,93.26,80.0,85.34,113.0,...,8.93,0.0,0.0,0.0,0.0,0.0,0.0,727.94,727.01,224.53
Total Bags,8696.87,9505.56,8145.35,5811.16,6183.95,6683.91,8318.86,6829.22,11388.36,8625.92,...,16762.57,13655.49,13964.33,13776.71,12693.57,13498.67,9264.84,9394.11,10969.54,12014.15
Small Bags,8603.62,9408.07,8042.21,5677.4,5986.26,6556.47,8196.81,6266.85,11104.53,8061.47,...,16510.32,13401.93,13698.27,13553.53,12437.35,13066.82,8940.04,9351.8,10919.54,11988.14
Large Bags,93.25,97.49,103.14,133.76,197.69,127.44,122.05,562.37,283.83,564.45,...,252.25,253.56,266.06,223.18,256.22,431.85,324.8,42.31,50.0,26.01


In [ ]:
#Sprawdzamy czy są jakieś puste zmienne
data.isnull().sum()

Unnamed: 0      0
Date            0
AveragePrice    0
Total Volume    0
4046            0
4225            0
4770            0
Total Bags      0
Small Bags      0
Large Bags      0
XLarge Bags     0
type            0
year            0
region          0
dtype: int64

In [ ]:
#Oddzielenie cen bezpośrednio z całego pliku data
y = data.AveragePrice
data.drop(['AveragePrice'], axis=1, inplace=True)

# Podział na zbiory uczące i testujące


---



In [ ]:
#podział na zbiór uczący i testujący

trainflights, testflights, ytrain, ytest = train_test_split(data, y, train_size=0.8,test_size=0.2, random_state=0)

#trainflights, ytrain - zbiór uczący - 80%
#testflights, ytest - zbiór testujący - 20%
#argumenty funkcji train_test_split(zbiór danych, zmienna docelowa, rozmiar zbioru uczącego, rozmiar zbioru testującego, podział determistyczny)


print("Rozmiar zbioru uczącego:", trainflights.shape)
print("Rozmiar zbioru testowego:", testflights.shape)

Rozmiar zbioru uczącego: (14599, 13)
Rozmiar zbioru testowego: (3650, 13)


In [ ]:
#analiza kolumn w zbiorze uczącym
#oddzielenie kolumn, które mają typ obiektowy
s = (trainflights.dtypes == 'object')
objCols = list(s[s].index)

#oodzielenie kolumn z typem float i int
n = (trainflights.dtypes == ('float64','int64'))
numCols = list(n[n].index)

print(objCols)
print(numCols)

['Date', 'type', 'region']
['Total Volume', '4046', '4225', '4770', 'Total Bags', 'Small Bags', 'Large Bags', 'XLarge Bags']


In [ ]:
#Przekształcanie danych obiektowych na kolumny binarne
from sklearn.preprocessing import OneHotEncoder
oneHot = OneHotEncoder(handle_unknown = 'ignore', sparse_output=False)
oneHottrain = pd.DataFrame(oneHot.fit_transform(trainflights[objCols]))
oneHottest = pd.DataFrame(oneHot.transform(testflights[objCols]))

In [ ]:
#Indeksy danych zakodowanych będą takie same jak indeksy oryginalnych danych
oneHottrain.index = trainflights.index
oneHottest.index = testflights.index 

In [ ]:
#kolumny, ktore nie sa danymi obiektowym z zbioru uczącego i testującego
cattraincol = trainflights.drop(objCols, axis=1)
cattestcol = testflights.drop(objCols, axis=1)

#Łączenie zbiorów 
trainflights = pd.concat([cattraincol, oneHottrain], axis=1)
testflights = pd.concat([cattestcol, oneHottest], axis=1)
#W rezultacie trainflights bedzie mialo kolumny numeryczne i zakodowane binarnie kolumny z obiektami

In [ ]:
#Przypisujemy wartości do

trainValues = trainflights.values
testValues = testflights.values

In [ ]:
#Skalowanie na podstawie wartości minimalnych i maksymalnych
#ma na celu doprowadzenie danych do określonego zakresu, najczęściej [0, 1]. Train i test są traktowane w ten sam sposób.

from sklearn.preprocessing import MinMaxScaler
minMax = MinMaxScaler()

trainflights = minMax.fit_transform(trainValues)
testflights = minMax.transform(testValues)

# Implementacja modeli

---



In [ ]:
#Procentowy błąd predykcji dla zestawu prognoz i zestawu rzeczywistego

from datetime import datetime, timedelta
def PercentError(preds, ytest):

  preds = np.ravel(preds)
  ytest = np.ravel(ytest)


  error = abs(preds - ytest)
  errorp = np.mean(100 - 100*(error/ytest))


  print('Dokładność wynosi: ', errorp)
  df_predicted_prices = pd.DataFrame(preds, columns=["Predicted Prices"])
  df_predicted_prices = df_predicted_prices.round(2)

  return df_predicted_prices

In [ ]:
#model sekwencyjny biblikoteki Keras
#dwie warstwy geste z 64 neuronami i funkcją aktywacji ReLU
#Warstwy geste - w pełni połączonymi warstwami,
#w których każdy neuron jest połączony z każdym neuronem w poprzedniej i następnej warstwie.
#+ warstwa wyjsciowa z jednym neuronem, ktora bedzie przewidywac wartosc wyjsciowa


def buildCompile():
  model = keras.Sequential([
      layers.Dense(64, activation = "relu"), #rectified linear unit #prosta, przyspiesza proces uczeniu, nie ma ograniczen zakresu wartosci wyjsciowych
      layers.Dense(64, activation = "relu"),
      layers.Dense(1)
 ])
  
  model.compile(loss='mean_absolute_error',
                optimizer=tf.keras.optimizers.Adam(0.001), #adaptive moment estimation 
                metrics=['mean_absolute_error', tf.keras.metrics.Recall()])
  
  return model

In [ ]:
def buildCompile2():
  model = keras.Sequential([
      layers.Dense(64, activation = "relu"),
      layers.Dense(32, activation = "sigmoid"), #wejscie przeksztalcone na zakres 0,1, idealne do problemow binarnych, gradient zanika przy duzycj wartosciach wejscioowe
      layers.Dense(1)
 ])
  
  model.compile(loss='mean_absolute_error',
                optimizer=tf.keras.optimizers.Adam(0.009),
                metrics=['mean_absolute_error', tf.keras.metrics.Recall()])
  
  return model

In [ ]:
def buildCompile3():
  model = keras.Sequential([
      layers.Dense(64, activation = "linear"), #liniowa, zachowuje wartosc wejscia jako wartosc wyjscia, dla modeli regresji liniowej, bo nie wprowadza nieliniowosci
      layers.Dense(64, activation = "linear"),
      layers.Dense(64, activation = "linear"),
      layers.Dense(1)
 ])
  
  model.compile(loss='mean_absolute_error',
                optimizer=tf.keras.optimizers.Adam(0.01),
                metrics=['mean_absolute_error', tf.keras.metrics.Recall()])
  
  return model

In [ ]:
def buildCompile4():
  model = keras.Sequential([
      layers.Dense(16, activation = "relu"),
      layers.Dense(16, activation = "linear"),
      layers.Dense(1)
 ])
  
  model.compile(loss='mean_absolute_error',
                optimizer=tf.keras.optimizers.Adam(0.01),
                metrics=['mean_absolute_error', tf.keras.metrics.Recall()])
  
  return model

In [ ]:
def buildCompile5():
  model = keras.Sequential([
      layers.Dense(64, activation = "sigmoid"),
      layers.Dense(32, activation = "relu"),
      layers.Dense(16, activation = "linear"),
      layers.Dense(1)
 ])
  
  model.compile(loss='mean_absolute_error',
                optimizer=tf.keras.optimizers.Adam(0.01),
                metrics=['mean_absolute_error', tf.keras.metrics.Recall()])
  
  return model

In [ ]:
# MODEL 1 - 150 EPOK

#Tworzenie i kompilowanie modelu
modelDNN = buildCompile();

#Trenujemy model
modelDNN.fit(trainflights, ytrain, epochs=150, validation_data=(testflights, ytest))

#Predykcja na danych testowych
predictions = modelDNN.predict(testflights)

PercentError(predictions, ytest)

# Odczyt wyników dla zbioru uczącego
train_results = modelDNN.evaluate(trainflights, ytrain)
print(f"Wyniki dla zbioru uczącego: {train_results}")

# Odczyt wyników dla zbioru testowego
test_results = modelDNN.evaluate(testflights, ytest)
print(f"Wyniki dla zbioru testowego: {test_results}")


Epoch 1/150
457/457 [==============================] - 2s 3ms/step - loss: 0.2327 - mean_absolute_error: 0.2327 - recall_6: 0.9647 - val_loss: 0.1588 - val_mean_absolute_error: 0.1588 - val_recall_6: 1.0000
Epoch 2/150
457/457 [==============================] - 1s 2ms/step - loss: 0.1520 - mean_absolute_error: 0.1520 - recall_6: 0.9999 - val_loss: 0.1538 - val_mean_absolute_error: 0.1538 - val_recall_6: 1.0000
Epoch 3/150
457/457 [==============================] - 1s 2ms/step - loss: 0.1420 - mean_absolute_error: 0.1420 - recall_6: 0.9999 - val_loss: 0.1467 - val_mean_absolute_error: 0.1467 - val_recall_6: 1.0000
Epoch 4/150
457/457 [==============================] - 1s 3ms/step - loss: 0.1331 - mean_absolute_error: 0.1331 - recall_6: 0.9997 - val_loss: 0.1429 - val_mean_absolute_error: 0.1429 - val_recall_6: 0.9997
Epoch 5/150
457/457 [==============================] - 2s 3ms/step - loss: 0.1219 - mean_absolute_error: 0.1219 - recall_6: 0.9999 - val_loss: 0.1339 - val_mean_absolute_er

In [ ]:
# MODEL 1 - 250 EPOK


#Tworzenie i kompilowanie modelu
modelDNN = buildCompile();

#Trenujemy model
modelDNN.fit(trainflights, ytrain, epochs=250, validation_data=(testflights, ytest))

#Predykcja na danych testowych
predictions = modelDNN.predict(testflights)

PercentError(predictions, ytest)

# Odczyt wyników dla zbioru uczącego
train_results = modelDNN.evaluate(trainflights, ytrain)
print(f"Wyniki dla zbioru uczącego: {train_results}")

# Odczyt wyników dla zbioru testowego
test_results = modelDNN.evaluate(testflights, ytest)
print(f"Wyniki dla zbioru testowego: {test_results}")

Epoch 1/250
457/457 [==============================] - 2s 3ms/step - loss: 0.2345 - mean_absolute_error: 0.2345 - recall_7: 0.9664 - val_loss: 0.1620 - val_mean_absolute_error: 0.1620 - val_recall_7: 1.0000
Epoch 2/250
457/457 [==============================] - 1s 2ms/step - loss: 0.1516 - mean_absolute_error: 0.1516 - recall_7: 0.9997 - val_loss: 0.1540 - val_mean_absolute_error: 0.1540 - val_recall_7: 1.0000
Epoch 3/250
457/457 [==============================] - 1s 2ms/step - loss: 0.1416 - mean_absolute_error: 0.1416 - recall_7: 0.9999 - val_loss: 0.1459 - val_mean_absolute_error: 0.1459 - val_recall_7: 1.0000
Epoch 4/250
457/457 [==============================] - 1s 2ms/step - loss: 0.1315 - mean_absolute_error: 0.1315 - recall_7: 0.9999 - val_loss: 0.1386 - val_mean_absolute_error: 0.1386 - val_recall_7: 1.0000
Epoch 5/250
457/457 [==============================] - 1s 2ms/step - loss: 0.1239 - mean_absolute_error: 0.1239 - recall_7: 0.9999 - val_loss: 0.1326 - val_mean_absolute_er

In [ ]:
# MODEL 2 - 150 EPOK

#Tworzenie i kompilowanie modelu
modelDNN = buildCompile2();

#Trenujemy model
modelDNN.fit(trainflights, ytrain, epochs=150, validation_data=(testflights, ytest))

#Predykcja na danych testowych
predictions = modelDNN.predict(testflights)

PercentError(predictions, ytest)

# Odczyt wyników dla zbioru uczącego
train_results = modelDNN.evaluate(trainflights, ytrain)
print(f"Wyniki dla zbioru uczącego: {train_results}")

# Odczyt wyników dla zbioru testowego
test_results = modelDNN.evaluate(testflights, ytest)
print(f"Wyniki dla zbioru testowego: {test_results}")

Epoch 1/150
457/457 [==============================] - 2s 3ms/step - loss: 0.1854 - mean_absolute_error: 0.1854 - recall_8: 0.9973 - val_loss: 0.1700 - val_mean_absolute_error: 0.1700 - val_recall_8: 1.0000
Epoch 2/150
457/457 [==============================] - 1s 2ms/step - loss: 0.1407 - mean_absolute_error: 0.1407 - recall_8: 0.9997 - val_loss: 0.1328 - val_mean_absolute_error: 0.1328 - val_recall_8: 0.9997
Epoch 3/150
457/457 [==============================] - 1s 2ms/step - loss: 0.1221 - mean_absolute_error: 0.1221 - recall_8: 0.9999 - val_loss: 0.1192 - val_mean_absolute_error: 0.1192 - val_recall_8: 0.9997
Epoch 4/150
457/457 [==============================] - 1s 2ms/step - loss: 0.1112 - mean_absolute_error: 0.1112 - recall_8: 0.9998 - val_loss: 0.1237 - val_mean_absolute_error: 0.1237 - val_recall_8: 1.0000
Epoch 5/150
457/457 [==============================] - 1s 2ms/step - loss: 0.1055 - mean_absolute_error: 0.1055 - recall_8: 0.9999 - val_loss: 0.1196 - val_mean_absolute_er

In [ ]:
# MODEL 2 - 350 EPOK

#Tworzenie i kompilowanie modelu
modelDNN = buildCompile2();

#Trenujemy model
modelDNN.fit(trainflights, ytrain, epochs=350, validation_data=(testflights, ytest))

#Predykcja na danych testowych
predictions = modelDNN.predict(testflights)

PercentError(predictions, ytest)

# Odczyt wyników dla zbioru uczącego
train_results = modelDNN.evaluate(trainflights, ytrain)
print(f"Wyniki dla zbioru uczącego: {train_results}")

# Odczyt wyników dla zbioru testowego
test_results = modelDNN.evaluate(testflights, ytest)
print(f"Wyniki dla zbioru testowego: {test_results}")

Epoch 1/350
457/457 [==============================] - 3s 4ms/step - loss: 0.1923 - mean_absolute_error: 0.1923 - recall_9: 0.9927 - val_loss: 0.1518 - val_mean_absolute_error: 0.1518 - val_recall_9: 1.0000
Epoch 2/350
457/457 [==============================] - 1s 2ms/step - loss: 0.1435 - mean_absolute_error: 0.1435 - recall_9: 0.9997 - val_loss: 0.1422 - val_mean_absolute_error: 0.1422 - val_recall_9: 1.0000
Epoch 3/350
457/457 [==============================] - 1s 2ms/step - loss: 0.1244 - mean_absolute_error: 0.1244 - recall_9: 0.9999 - val_loss: 0.1257 - val_mean_absolute_error: 0.1257 - val_recall_9: 0.9997
Epoch 4/350
457/457 [==============================] - 1s 2ms/step - loss: 0.1120 - mean_absolute_error: 0.1120 - recall_9: 0.9999 - val_loss: 0.1177 - val_mean_absolute_error: 0.1177 - val_recall_9: 0.9997
Epoch 5/350
457/457 [==============================] - 1s 2ms/step - loss: 0.1051 - mean_absolute_error: 0.1051 - recall_9: 0.9998 - val_loss: 0.1172 - val_mean_absolute_er

In [ ]:
# MODEL 3 - 150 EPOK

#Tworzenie i kompilowanie modelu
modelDNN = buildCompile3();

#Trenujemy model
modelDNN.fit(trainflights, ytrain, epochs=150, validation_data=(testflights, ytest))

#Predykcja na danych testowych
predictions = modelDNN.predict(testflights)

PercentError(predictions, ytest)

# Odczyt wyników dla zbioru uczącego
train_results = modelDNN.evaluate(trainflights, ytrain)
print(f"Wyniki dla zbioru uczącego: {train_results}")

# Odczyt wyników dla zbioru testowego
test_results = modelDNN.evaluate(testflights, ytest)
print(f"Wyniki dla zbioru testowego: {test_results}")

Epoch 1/150
457/457 [==============================] - 3s 4ms/step - loss: 0.2247 - mean_absolute_error: 0.2247 - recall_10: 0.9938 - val_loss: 0.1909 - val_mean_absolute_error: 0.1909 - val_recall_10: 1.0000
Epoch 2/150
457/457 [==============================] - 1s 3ms/step - loss: 0.1866 - mean_absolute_error: 0.1866 - recall_10: 0.9998 - val_loss: 0.1817 - val_mean_absolute_error: 0.1817 - val_recall_10: 1.0000
Epoch 3/150
457/457 [==============================] - 1s 3ms/step - loss: 0.1811 - mean_absolute_error: 0.1811 - recall_10: 0.9997 - val_loss: 0.1783 - val_mean_absolute_error: 0.1783 - val_recall_10: 1.0000
Epoch 4/150
457/457 [==============================] - 1s 3ms/step - loss: 0.1768 - mean_absolute_error: 0.1768 - recall_10: 1.0000 - val_loss: 0.1784 - val_mean_absolute_error: 0.1784 - val_recall_10: 1.0000
Epoch 5/150
457/457 [==============================] - 2s 3ms/step - loss: 0.1787 - mean_absolute_error: 0.1787 - recall_10: 0.9991 - val_loss: 0.1885 - val_mean_ab

In [ ]:
# MODEL 3 - 350 EPOK

#Tworzenie i kompilowanie modelu
modelDNN = buildCompile3();

#Trenujemy model
modelDNN.fit(trainflights, ytrain, epochs=350, validation_data=(testflights, ytest))

#Predykcja na danych testowych
predictions = modelDNN.predict(testflights)

PercentError(predictions, ytest)

# Odczyt wyników dla zbioru uczącego
train_results = modelDNN.evaluate(trainflights, ytrain)
print(f"Wyniki dla zbioru uczącego: {train_results}")

# Odczyt wyników dla zbioru testowego
test_results = modelDNN.evaluate(testflights, ytest)
print(f"Wyniki dla zbioru testowego: {test_results}")

Epoch 1/350
457/457 [==============================] - 3s 3ms/step - loss: 0.2124 - mean_absolute_error: 0.2124 - recall_11: 0.9968 - val_loss: 0.1776 - val_mean_absolute_error: 0.1776 - val_recall_11: 1.0000
Epoch 2/350
457/457 [==============================] - 2s 4ms/step - loss: 0.1877 - mean_absolute_error: 0.1877 - recall_11: 0.9998 - val_loss: 0.1764 - val_mean_absolute_error: 0.1764 - val_recall_11: 1.0000
Epoch 3/350
457/457 [==============================] - 1s 3ms/step - loss: 0.1808 - mean_absolute_error: 0.1808 - recall_11: 1.0000 - val_loss: 0.1782 - val_mean_absolute_error: 0.1782 - val_recall_11: 1.0000
Epoch 4/350
457/457 [==============================] - 2s 4ms/step - loss: 0.1779 - mean_absolute_error: 0.1779 - recall_11: 0.9998 - val_loss: 0.2103 - val_mean_absolute_error: 0.2103 - val_recall_11: 0.9907
Epoch 5/350
457/457 [==============================] - 1s 3ms/step - loss: 0.1782 - mean_absolute_error: 0.1782 - recall_11: 0.9996 - val_loss: 0.1764 - val_mean_ab

In [ ]:
# MODEL 4 - 150 EPOK

#Tworzenie i kompilowanie modelu
modelDNN = buildCompile4();

#Trenujemy model
modelDNN.fit(trainflights, ytrain, epochs=150, validation_data=(testflights, ytest))

#Predykcja na danych testowych
predictions = modelDNN.predict(testflights)

PercentError(predictions, ytest)

# Odczyt wyników dla zbioru uczącego
train_results = modelDNN.evaluate(trainflights, ytrain)
print(f"Wyniki dla zbioru uczącego: {train_results}")

# Odczyt wyników dla zbioru testowego
test_results = modelDNN.evaluate(testflights, ytest)
print(f"Wyniki dla zbioru testowego: {test_results}")

Epoch 1/150
457/457 [==============================] - 3s 4ms/step - loss: 0.2212 - mean_absolute_error: 0.2212 - recall_12: 0.9943 - val_loss: 0.1945 - val_mean_absolute_error: 0.1945 - val_recall_12: 0.9984
Epoch 2/150
457/457 [==============================] - 1s 3ms/step - loss: 0.1884 - mean_absolute_error: 0.1884 - recall_12: 0.9993 - val_loss: 0.1863 - val_mean_absolute_error: 0.1863 - val_recall_12: 0.9995
Epoch 3/150
457/457 [==============================] - 1s 3ms/step - loss: 0.1801 - mean_absolute_error: 0.1801 - recall_12: 0.9997 - val_loss: 0.1920 - val_mean_absolute_error: 0.1920 - val_recall_12: 1.0000
Epoch 4/150
457/457 [==============================] - 2s 4ms/step - loss: 0.1796 - mean_absolute_error: 0.1796 - recall_12: 0.9996 - val_loss: 0.1938 - val_mean_absolute_error: 0.1938 - val_recall_12: 0.9953
Epoch 5/150
457/457 [==============================] - 2s 3ms/step - loss: 0.1803 - mean_absolute_error: 0.1803 - recall_12: 0.9995 - val_loss: 0.1922 - val_mean_ab

In [ ]:
# MODEL 4 - 350 EPOK

#Tworzenie i kompilowanie modelu
modelDNN = buildCompile4();

#Trenujemy model
modelDNN.fit(trainflights, ytrain, epochs=350, validation_data=(testflights, ytest))

#Predykcja na danych testowych
predictions = modelDNN.predict(testflights)

PercentError(predictions, ytest)

# Odczyt wyników dla zbioru uczącego
train_results = modelDNN.evaluate(trainflights, ytrain)
print(f"Wyniki dla zbioru uczącego: {train_results}")

# Odczyt wyników dla zbioru testowego
test_results = modelDNN.evaluate(testflights, ytest)
print(f"Wyniki dla zbioru testowego: {test_results}")

Epoch 1/350
457/457 [==============================] - 4s 6ms/step - loss: 0.1986 - mean_absolute_error: 0.1986 - recall: 0.9930 - val_loss: 0.2015 - val_mean_absolute_error: 0.2015 - val_recall: 1.0000
Epoch 2/350
457/457 [==============================] - 1s 3ms/step - loss: 0.1614 - mean_absolute_error: 0.1614 - recall: 1.0000 - val_loss: 0.1607 - val_mean_absolute_error: 0.1607 - val_recall: 1.0000
Epoch 3/350
457/457 [==============================] - 1s 2ms/step - loss: 0.1487 - mean_absolute_error: 0.1487 - recall: 1.0000 - val_loss: 0.1463 - val_mean_absolute_error: 0.1463 - val_recall: 1.0000
Epoch 4/350
457/457 [==============================] - 1s 3ms/step - loss: 0.1391 - mean_absolute_error: 0.1391 - recall: 1.0000 - val_loss: 0.1425 - val_mean_absolute_error: 0.1425 - val_recall: 1.0000
Epoch 5/350
457/457 [==============================] - 2s 3ms/step - loss: 0.1331 - mean_absolute_error: 0.1331 - recall: 1.0000 - val_loss: 0.1372 - val_mean_absolute_error: 0.1372 - val_

In [ ]:
# MODEL 5 - 150 EPOK

#Tworzenie i kompilowanie modelu
modelDNN = buildCompile5();

#Trenujemy model
modelDNN.fit(trainflights, ytrain, epochs=150, validation_data=(testflights, ytest))

#Predykcja na danych testowych
predictions = modelDNN.predict(testflights)

PercentError(predictions, ytest)

# Odczyt wyników dla zbioru uczącego
train_results = modelDNN.evaluate(trainflights, ytrain)
print(f"Wyniki dla zbioru uczącego: {train_results}")

# Odczyt wyników dla zbioru testowego
test_results = modelDNN.evaluate(testflights, ytest)
print(f"Wyniki dla zbioru testowego: {test_results}")

Epoch 1/150
457/457 [==============================] - 2s 3ms/step - loss: 0.1940 - mean_absolute_error: 0.1940 - recall_13: 0.9976 - val_loss: 0.1727 - val_mean_absolute_error: 0.1727 - val_recall_13: 1.0000
Epoch 2/150
457/457 [==============================] - 1s 2ms/step - loss: 0.1725 - mean_absolute_error: 0.1725 - recall_13: 0.9999 - val_loss: 0.1647 - val_mean_absolute_error: 0.1647 - val_recall_13: 1.0000
Epoch 3/150
457/457 [==============================] - 1s 3ms/step - loss: 0.1630 - mean_absolute_error: 0.1630 - recall_13: 0.9999 - val_loss: 0.1825 - val_mean_absolute_error: 0.1825 - val_recall_13: 1.0000
Epoch 4/150
457/457 [==============================] - 2s 4ms/step - loss: 0.1564 - mean_absolute_error: 0.1564 - recall_13: 0.9999 - val_loss: 0.1578 - val_mean_absolute_error: 0.1578 - val_recall_13: 1.0000
Epoch 5/150
457/457 [==============================] - 1s 3ms/step - loss: 0.1535 - mean_absolute_error: 0.1535 - recall_13: 0.9997 - val_loss: 0.1519 - val_mean_ab

In [ ]:
# MODEL 5 - 350 EPOK

#Tworzenie i kompilowanie modelu
modelDNN = buildCompile5();

#Trenujemy model
modelDNN.fit(trainflights, ytrain, epochs=350, validation_data=(testflights, ytest))

#Predykcja na danych testowych
predictions = modelDNN.predict(testflights)

PercentError(predictions, ytest)

# Odczyt wyników dla zbioru uczącego
train_results = modelDNN.evaluate(trainflights, ytrain)
print(f"Wyniki dla zbioru uczącego: {train_results}")

# Odczyt wyników dla zbioru testowego
test_results = modelDNN.evaluate(testflights, ytest)
print(f"Wyniki dla zbioru testowego: {test_results}")

Epoch 1/350
457/457 [==============================] - 2s 3ms/step - loss: 0.1924 - mean_absolute_error: 0.1924 - recall_14: 0.9978 - val_loss: 0.1610 - val_mean_absolute_error: 0.1610 - val_recall_14: 1.0000
Epoch 2/350
457/457 [==============================] - 1s 3ms/step - loss: 0.1629 - mean_absolute_error: 0.1629 - recall_14: 1.0000 - val_loss: 0.1554 - val_mean_absolute_error: 0.1554 - val_recall_14: 1.0000
Epoch 3/350
457/457 [==============================] - 2s 4ms/step - loss: 0.1561 - mean_absolute_error: 0.1561 - recall_14: 1.0000 - val_loss: 0.1572 - val_mean_absolute_error: 0.1572 - val_recall_14: 1.0000
Epoch 4/350
457/457 [==============================] - 1s 2ms/step - loss: 0.1528 - mean_absolute_error: 0.1528 - recall_14: 1.0000 - val_loss: 0.1484 - val_mean_absolute_error: 0.1484 - val_recall_14: 1.0000
Epoch 5/350
457/457 [==============================] - 1s 2ms/step - loss: 0.1504 - mean_absolute_error: 0.1504 - recall_14: 1.0000 - val_loss: 0.1466 - val_mean_ab